In [1]:
import cv2
import os
import numpy as np
import matplotlib as plt
from glob import glob
import zipfile
from datetime import datetime

%matplotlib inline

In [2]:
# caso queira salvar ou carregar o modelo
import pickle
nome_medelo = "model_classificar_dog_cat.mod"

### Importar Dados e tratar os dados

In [3]:
# lista arquivos por formato
def filtrar_arquivo(tipo):
    path = './'
    lst_arq = os.listdir(path) # listando novamente o diretório para verificar arquivos csv
    return [arq for arq in lst_arq if arq[-3:] == tipo]

# Separa arquivo zip caso haja outros tipos de arquivo na pasta
lst_zip = filtrar_arquivo('zip')

# extrair arquivos do zip para trabalhar
for zp in lst_zip:
    fzp = zipfile.ZipFile(zp)
    fzp.extractall('./') 
    fzp.close()

In [4]:
# dog = 0
# cat = 1
class_name = ['dog','cat']

In [5]:
dir_img_train = 'dataset/train/*.jpg'
dir_img_test = 'dataset/test/*.jpg'

In [6]:
def convert_img_array(lst):
    """
    lista diretorios com imagens e convertem array
    
    """
    lst_img, lst_label=[],[]
    for d in glob(lst):
        lst_img.append(cv2.imread(d))
        if 'dog' in d:
            lst_label.append('0')
        elif 'cat' in d:
            lst_label.append('1')
    return np.array(lst_img), np.array(lst_label)

In [7]:
# gerando dados de trainos e test
train_img, train_label = convert_img_array(dir_img_train)
test_img, test_label = convert_img_array(dir_img_test)

In [8]:
train_img.shape

(8000, 32, 32, 3)

In [9]:
test_label.shape

(2000,)

In [10]:
train_img.shape

(8000, 32, 32, 3)

In [11]:
train_label.shape

(8000,)

In [12]:
# escalar as imagens para que o valor esteja entre 0 e 1
train_img = train_img / 255.0
test_img = test_img / 255.0

### Classificação com Tensorflow

Melhor acurácia utilizando optimizer('sgd') = 0.66

In [13]:
# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras

In [14]:
# criar o modelo
model = keras.Sequential([
    keras.layers.Flatten(input_shape = (32, 32, 3)),
    keras.layers.Dense(128, activation = tf.nn.relu),
    keras.layers.Dense(128, activation = tf.nn.relu),
    keras.layers.Dense(128, activation = tf.nn.relu),
    keras.layers.Dense(2, activation = tf.nn.softmax)
])

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
# compilar modelo
model.compile(optimizer='sgd', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
             )

In [ ]:
start = datetime.now()
# treinar modelo
model.fit(train_img, train_label, epochs=100000)
print('')
print('Tempo decorrido para treinar o modelo: ', datetime.now() - start)

Epoch 1/100000
8000/8000 [==============================] - 2s 257us/sample - loss: 0.6862 - acc: 0.5497
Epoch 2/100000
8000/8000 [==============================] - 2s 238us/sample - loss: 0.6684 - acc: 0.5885
Epoch 3/100000
8000/8000 [==============================] - 2s 249us/sample - loss: 0.6594 - acc: 0.6041
Epoch 4/100000
8000/8000 [==============================] - 2s 235us/sample - loss: 0.6536 - acc: 0.6143
Epoch 5/100000
8000/8000 [==============================] - 2s 236us/sample - loss: 0.6455 - acc: 0.6254
Epoch 6/100000
8000/8000 [==============================] - 2s 241us/sample - loss: 0.6406 - acc: 0.6281
Epoch 7/100000
8000/8000 [==============================] - 2s 248us/sample - loss: 0.6356 - acc: 0.6331
Epoch 8/100000
8000/8000 [==============================] - 2s 243us/sample - loss: 0.6287 - acc: 0.6464
Epoch 9/100000
8000/8000 [==============================] - 2s 243us/sample - loss: 0.6245 - acc: 0.6486
Epoch 10/100000
8000/8000 [============================

In [55]:
# eficiência do modelo
test_loss, test_acc = model.evaluate(test_img, test_label)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

2000/2000 [==============================] - 0s 246us/sample - loss: 2.2670 - acc: 0.6585
Test accuracy: 0.6585
Test loss: 2.2669596309661864


In [56]:
# previsoes
predict = model.predict(test_img)
predict

array([[4.0509422e-05, 9.9995947e-01],
       [4.4648236e-04, 9.9955350e-01],
       [4.8662182e-06, 9.9999511e-01],
       ...,
       [5.9292514e-02, 9.4070745e-01],
       [9.9450266e-01, 5.4972898e-03],
       [6.5560102e-02, 9.3443990e-01]], dtype=float32)

In [58]:
# previsão é um vetor de 2 números onde descreve a confiança que teve para cada resultado
p = 91

print('Predito:', class_name[np.argmax(predict[p])])
print('Real:', class_name[int(test_label[p])])

Predito: cat
Real: cat


### Classificação com Machine Learning

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
# padroniza a entrada com esse reshape
# convert de 3d para 2d
def _reshape(ds):
    return ds.reshape(len(ds),-1)

In [14]:
x_train = _reshape(train_img)
y_test = _reshape(test_img)

#### Modelo SVM 

Melhor acurácia = 0.62

In [ ]:
from sklearn.svm import SVC

In [ ]:
# accuracy testada do modelo
# 'linear' =  0.5935
# 'poly' = 0.566
# 'rbf' = 0.6195
#'sigmoid' = 0.5885

model_svm = SVC(kernel='rbf')
model_svm.fit(x_train, train_label)

predicted = model_svm.predict(y_test)
accuracy_score(test_label, predicted)

#### Regressão Logistica

Melhor acurácia = 0.60 com 'sag'

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_log = LogisticRegression(solver='sag')

In [ ]:
model_log.fit(x_train, train_label)
pred_log1 = model_log.predict(y_test)
accuracy_score(test_label, pred_log1)

#### Floresta aleatória (Random Florest)

Melhor acurácia = 0.58

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
model_rd = RandomForestClassifier()

In [17]:
model_rd.fit(x_train, train_label)
pred_rd = model_rd.predict(y_test)
accuracy_score(test_label, pred_rd)

d:\__venvs_projects\deep_learn\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5925

#### Naives Bayes

Melhor acurácia = 0.59

In [31]:
from sklearn.naive_bayes import GaussianNB

In [32]:
model_nb = GaussianNB()

In [33]:
model_nb.fit(x_train, train_label)
pred_nb = model_rd.predict(y_test)
accuracy_score(test_label, pred_nb)

0.5925

#### K-Nearest Neighbors (KNN)

Melhor acurácia = 0.58

In [34]:
from sklearn.neighbors import KNeighborsClassifier

In [35]:
model_knn = KNeighborsClassifier()

In [36]:
model_knn.fit(x_train, train_label)
pred_knn = model_knn.predict(y_test)
accuracy_score(test_label, pred_knn)

0.5825